In [67]:
#Įsikeliame reikalingas bibliotekas
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

import keras
import numpy as np
import keras.backend as K
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.models import load_model
from keras import regularizers, optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_normal, RandomNormal, Zeros
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
import os

In [68]:
# funkcija, kuri grąžina analogišką paveikslėlį kaip mokymo klasėje
def get_image_for_model(image_path, image_size = 200):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (image_size, image_size))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.expand_dims(img, axis=-1) # pridedama papildoma dimensija (200, 200, 1)
    img = img / 255
    img = np.expand_dims(img, axis=0) # pridedama papildoma dimensija (1, 200, 200, 1)
    return img

In [69]:
# analogiškas mokymo modelis, naudojama svorių užkrovimui
def create_model(s = 2, weight_decay = 1e-2, act="relu", input_shape=(200, 200, 1)):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal(), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2)) # mokantis ismeta nurodyta skaiciu neuronu. (leidzia isvengti overfitting)
    
    # Block 2
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 3
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 4
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    # First Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    
    # Block 5
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 6
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 7
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    # Second Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    
    # Block 8
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 9
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    # Third Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    
    # Block 10
    model.add(Conv2D(512, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    # Block 11  
    model.add(Conv2D(2048, (1,1), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 12  
    model.add(Conv2D(256, (1,1), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fourth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))


    # Block 13
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fifth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    # Final Classifier
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [70]:
model = create_model()
model.load_weights('simplenet_trained_model_202012061141.h5')

# Test
img = get_image_for_model(r"C:\Users\daimzn\Asmeniniai\Schemos\Schemos\nzn\1980_EU-CR-2000028_4_PDF.png")
model.predict(img)[0][0]

In [66]:
demo_dir = r'C:\Users\daimzn\Asmeniniai\Schemos\Demo'
schemos_dir = os.path.join(demo_dir, 'schemos')
ne_schemos_dir = os.path.join(demo_dir, 'ne_schemos')

if not os.path.isdir(schemos_dir):
    os.mkdir(schemos_dir)
if not os.path.isdir(ne_schemos_dir):
    os.mkdir(ne_schemos_dir)
    
for i, demo_file in enumerate(os.listdir(demo_dir)):
    if demo_file.lower().endswith('png'):
        demo_fp = os.path.join(demo_dir, demo_file)
        img = get_image_for_model(demo_fp)
        pred = model.predict(img)[0][0]
        # jeigu 90% tikslumu identifikuojama, jog tai schema, tuomet perkeliama į schemų katalogą
        if pred > 0.9:
            os.rename(demo_fp, os.path.join(schemos_dir, demo_file))
        else:
            os.rename(demo_fp, os.path.join(ne_schemos_dir, demo_file))